# Imports

In [113]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

import tqdm
import matplotlib.pyplot as plt
import time
import os
import copy


# Setting device for the computation

In [114]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Resizing all the images to the same size

In [115]:
train_transform = transforms.Compose([
        #TODO:think a better transformation pipeline
        #naive transformation
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ])
test_transform = transforms.Compose([
        #TODO:think a better transformation pipeline
        #naive transformation
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ])

# Loading the train dataset

In [116]:
train_dir='./dataset/GTSRB/train'

train_dataset = datasets.ImageFolder(train_dir,train_transform)

train_loader = DataLoader(train_dataset, batch_size=1000,shuffle=True, num_workers=4)

train_total_steps = len(train_loader)
train_size = len(train_dataset)
class_names = train_dataset.classes

print('Total steps for training:',train_total_steps)
print('Train size:',train_size)
print('Class names:',class_names)

Total steps for training: 40
Train size: 39209
Class names: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Loading the test dataset

In [127]:
test_dir='./dataset/GTSRB/test'

test_dataset = datasets.ImageFolder(test_dir,test_transform)

test_loader = DataLoader(test_dataset, shuffle=True, num_workers=0)

test_size = len(test_dataset)
class_names = test_dataset.classes

print('Test size:',train_size)
print('Class names:',class_names)

Test size: 39209
Class names: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Defining the training phase

In [118]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        #iterates over the batches
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
    
            # forward pass
            outputs = model(images) #compute predictions
            _, preds = torch.max(outputs, 1) #maximum value along dimension 1
            loss = criterion(outputs, labels)

            # backward + optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #prints the stats every 20 steps (20 batches performed)
            if (i+1) % 10 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{train_total_steps}], Loss: {loss.item():.4f}')
    
            # statistics
            running_loss += loss.item() * images.size(0) # scalar loss * n.of samples in the batch: necessary to average the loss over the batch
            running_corrects += torch.sum(preds == labels.data)
            
            #to update weights
            # scheduler.step()
    
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
    
        print('{} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(
            'Train phase - ',epoch, epoch_loss, epoch_acc))
    
    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return model

# Defining the model using ResNet18 as backbone

In [119]:
from torchvision.models import ResNet18_Weights

#### ConvNet as fixed feature extractor ####
# freeze all the network parameters except the final layer settings, to not compute the gradients backward
model_conv = torchvision.models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
# Define the layers you want to add
additional_layers = nn.Sequential(
    # nn.Linear(num_ftrs, 100),  # Example additional layer with 100 units
    # nn.ReLU(),                  # Activation function
    # nn.Linear(100, 100),         # Another layer with 50 units
    # nn.ReLU(),
    nn.Linear(num_ftrs, 43)
)
model_conv.fc = additional_layers


model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized
optimizer_conv = torch.optim.Adam(model_conv.fc.parameters(), lr=0.05)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                        exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
Epoch [1/20], Step [10/40], Loss: 24.4728
Epoch [1/20], Step [20/40], Loss: 11.7694
Epoch [1/20], Step [30/40], Loss: 5.0185
Epoch [1/20], Step [40/40], Loss: 3.2709
Train phase -  Epoch 0 Loss: 13.3178 Acc: 0.3215
Epoch 1/19
----------
Epoch [2/20], Step [10/40], Loss: 1.7734
Epoch [2/20], Step [20/40], Loss: 1.5437
Epoch [2/20], Step [30/40], Loss: 1.4148
Epoch [2/20], Step [40/40], Loss: 1.6063
Train phase -  Epoch 1 Loss: 1.6797 Acc: 0.6582
Epoch 2/19
----------
Epoch [3/20], Step [10/40], Loss: 1.0736
Epoch [3/20], Step [20/40], Loss: 0.9824
Epoch [3/20], Step [30/40], Loss: 0.9941
Epoch [3/20], Step [40/40], Loss: 1.2210
Train phase -  Epoch 2 Loss: 0.9993 Acc: 0.7278
Epoch 3/19
----------
Epoch [4/20], Step [10/40], Loss: 0.8382
Epoch [4/20], Step [20/40], Loss: 0.7618
Epoch [4/20], Step [30/40], Loss: 0.7704
Epoch [4/20], Step [40/40], Loss: 0.7365
Train phase -  Epoch 3 Loss: 0.8145 Acc: 0.7601
Epoch 4/19
----------
Epoch [5/20], Step [10/40], Loss: 0.761

# Saving the trained model

In [120]:
print('Finished Training')
os.makedirs('./models', exist_ok=True)
PATH = './models/trained_model.pth'
torch.save(model_conv, PATH)

Finished Training


# Loading the model

In [121]:
trained_model=torch.load('./models/trained_model.pth',map_location=device)

# Evaluating the model

In [131]:
def test_model(model, dataloader, dataset_size):
    model.eval()  # Set model to evaluation mode
    
    running_corrects = 0

    with torch.no_grad():  # Disable gradient calculation
        for i, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
    
            # Statistics
            running_corrects += torch.sum(preds == labels.data)
            if (i+1) % 50 == 0:
                print ( f'Evaluating: [{i+1}/{test_size}]')

    # Calculate accuracy
    test_acc = running_corrects.double() / dataset_size

    print('Test Acc: {:.4f}'.format( test_acc))
    

In [132]:
test_model(trained_model, test_loader, test_size)

Evaluating: [50/12630]
Evaluating: [100/12630]
Evaluating: [150/12630]
Evaluating: [200/12630]
Evaluating: [250/12630]
Evaluating: [300/12630]
Evaluating: [350/12630]
Evaluating: [400/12630]
Evaluating: [450/12630]
Evaluating: [500/12630]
Evaluating: [550/12630]
Evaluating: [600/12630]
Evaluating: [650/12630]
Evaluating: [700/12630]
Evaluating: [750/12630]
Evaluating: [800/12630]
Evaluating: [850/12630]
Evaluating: [900/12630]
Evaluating: [950/12630]
Evaluating: [1000/12630]
Evaluating: [1050/12630]
Evaluating: [1100/12630]
Evaluating: [1150/12630]
Evaluating: [1200/12630]
Evaluating: [1250/12630]
Evaluating: [1300/12630]
Evaluating: [1350/12630]
Evaluating: [1400/12630]
Evaluating: [1450/12630]
Evaluating: [1500/12630]
Evaluating: [1550/12630]
Evaluating: [1600/12630]
Evaluating: [1650/12630]
Evaluating: [1700/12630]
Evaluating: [1750/12630]
Evaluating: [1800/12630]
Evaluating: [1850/12630]
Evaluating: [1900/12630]
Evaluating: [1950/12630]
Evaluating: [2000/12630]
Evaluating: [2050/12